In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import binom

### Question 1

In [2]:
priceCL = 300 # low price for coach
priceCH = 350 # high price for coach
priceFL = 425 # low price for first class
priceFH = 500 # high price for first class

pCL = [0.35,0.65] # demand probabilities for coach low price
pCH = [0.7,0.3] # demand probabilities for coach high price
pFL = [0.92,0.08] # demand probabilities for first class low price
pFH = [0.96,0.04] # demand probabilities for first class high price

bumpL = 50 # cost to bump passenger to first class
bumpH = 425 # cost to bump passenger off plane
showC = 0.95 # probability of coach passenger showing up
showF = 0.97 # probability of first class passenger showing up

delta = 1/(1+0.15/365) # discount rate

C = 100 # number of seats for coach
O = 5 # number of seats for overbook
F = 20 # number of seats for first class
T = 365 # days until takeoff

cValues = np.arange(C+O+1) # all possible number of coach seats left
fValues = np.arange(F+1) # all possible number of first class seats left
tValues = np.arange(T+1) # all possible days until takeoff

cN = len(cValues) # count possible state values for coach seats
fN = len(fValues) # count possible state values for first class seats
tN = len(tValues) # count possible state values for time

In [3]:
V = np.zeros((cN,fN,tN)) # initialize value function
U = np.zeros((cN,fN,tN)) # initialize optimal choice variable

In [4]:
# boundary/terminal condition
V[:,:,tN-1] = 0 # when the flight takes off you can't make any more money
U[:,:,tN-1] = 0 # 0 is the value for no tickets for sale

for c in range(cN):
    for f in range(fN):
        if c > 100 and f == 20:
            cost = -(c-100)*(bumpH)
            V[c,f,tN-1] = cost * binom.pmf(c, c, showC) * binom.pmf(f, f, showF)
        elif c > 100 and f < 20:
            space = 20-f
            if c-100 < space:
                cost = -(c-100)*bumpL
            else:
                cost = -(space)*bumpL -((c-100)-space)*bumpH
            V[c,f,tN-1] = cost * binom.pmf(c, c, showC) * binom.pmf(f, f, showF)
        else:
            V[c,f,tN-1] = 0

In [5]:
for t in reversed(range(tN-1)): # loop backwards in time
    for c in range(cN): # loop over all possible coach seat values
        for f in range(fN): # loop over all possible first class seat values
        
            if c == 0 and f == 0: # if the coach seats and first class seats are full (0 seats left)
                V[c,f,t] = 0 # if so, you can't make any more money
                U[c,f,t] = 0 # no tickets for sale
                
            elif c == 0 and f != 0:
                FL = (priceFL*pFL[1]) + delta*(pFL[1]*V[c,f-1,t+1] + pFL[0]*V[c,f,t+1])
                FH = (priceFH*pFH[1]) + delta*(pFH[1]*V[c,f-1,t+1] + pFH[0]*V[c,f,t+1])
                V[c,f,t] = max(FL, FH)
                U[c,f,t] = np.argmax([FL, FH]) + 1 # choice of price: 1 means (0, L), 2 means (0, H)
                
            elif c != 0 and f == 0:
                CL = (priceCL*pCL[1]) + delta*(pCL[1]*V[c-1,f,t+1] + pCL[0]*V[c,f,t+1])
                CH = (priceCH*pCH[1]) + delta*(pCH[1]*V[c-1,f,t+1] + pCH[0]*V[c,f,t+1])
                V[c,f,t] = max(CL, CH)
                U[c,f,t] = np.argmax([CL, CH]) + 3 # choice of price: 3 means (L, 0), 4 means (H, 0)
            
            else:
                CLFL = ((priceCL+priceFL)*pCL[1]*pFL[1]) + ((0+priceFL)*pCL[0]*pFL[1]) + \
                       ((priceCL+0)*pCL[1]*pFL[0]) + ((0+0)*pCL[0]*pFL[0]) + \
                        delta*((pCL[1]+pFL[1])*V[c-1,f-1,t+1]) + delta*((pCL[0]+pFL[1])*V[c,f-1,t+1]) + \
                        delta*((pCL[1]+pFL[0])*V[c-1,f,t+1]) + delta*((pCL[0]+pFL[0])*V[c,f,t+1]) 
                
                CLFH = ((priceCL+priceFH)*pCL[1]*pFH[1]) + ((0+priceFH)*pCL[0]*pFH[1]) + \
                       ((priceCL+0)*pCL[1]*pFH[0]) + ((0+0)*pCL[0]*pFH[0]) + \
                        delta*((pCL[1]+pFH[1])*V[c-1,f-1,t+1]) + delta*((pCL[0]+pFH[1])*V[c,f-1,t+1]) + \
                        delta*((pCL[1]+pFH[0])*V[c-1,f,t+1]) + delta*((pCL[0]+pFH[0])*V[c,f,t+1])  
                
                CHFL = ((priceCH+priceFL)*pCH[1]*pFL[1]) + ((0+priceFL)*pCH[0]*pFL[1]) + \
                       ((priceCH+0)*pCH[1]*pFL[0]) + ((0+0)*pCH[0]*pFL[0]) + \
                        delta*((pCH[1]+pFL[1])*V[c-1,f-1,t+1]) + delta*((pCH[0]+pFL[1])*V[c,f-1,t+1]) + \
                        delta*((pCH[1]+pFL[0])*V[c-1,f,t+1]) + delta*((pCH[0]+pFL[0])*V[c,f,t+1]) 
                
                CHFH = ((priceCH+priceFH)*pCH[1]*pFH[1]) + ((0+priceFH)*pCH[0]*pFH[1]) + \
                       ((priceCH+0)*pCH[1]*pFH[0]) + ((0+0)*pCH[0]*pFH[0]) + \
                        delta*((pCH[1]+pFH[1])*V[c-1,f-1,t+1]) + delta*((pCH[0]+pFH[1])*V[c,f-1,t+1]) + \
                        delta*((pCH[1]+pFH[0])*V[c-1,f,t+1]) + delta*((pCH[0]+pFH[0])*V[c,f,t+1]) 

                V[c,f,t] = max([CLFL,CLFH,CHFL,CHFH]) # value funciton maximizes expected revenue
                U[c,f,t] = np.argmax([CLFL,CLFH,CHFL,CHFH]) +5 # choice of price: 5 means LL, 6 LH, 7 means HL, 8 means HH

In [6]:
c = cN-1 # on the first day all coach seats are available
f = fN-1 # on the first day all first class seats are available
revenue_vec = np.zeros(tN-1)
price_vec = np.zeros(tN-1)
total_revenue = 0

for t in range(tN-1): # loop forward in time...don't need last time period since we can't sell anything then
    opt_price = U[c,f,t] # how much should we charge
    price_vec[t] = opt_price
    
    if opt_price == 1:
        expected_revenue = priceFL*pFL[1]
    elif opt_price == 2: 
        expected_revenue = priceFH*pFH[1]
    elif opt_price == 3: 
        expected_revenue = priceCL*pCL[1]
    elif opt_price == 4: 
        expected_revenue = priceCH*pCH[1]  
    elif opt_price == 5: 
        expected_revenue = priceCL*pCL[1] + priceFL*pFL[1]
    elif opt_price == 6: 
        expected_revenue = priceCL*pCL[1] + priceFH*pFH[1]
    elif opt_price == 7: 
        expected_revenue = priceCH*pCH[1] + priceFL*pFL[1]
    elif opt_price == 8: 
        expected_revenue = priceCH*pCH[1] + priceFH*pFH[1]
    else: 
        expected_revenue = 0
        
    total_revenue += expected_revenue*delta**(t-1)

    revenue_vec[t] = total_revenue
